In [ ]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff xxhash pyyaml geopandas dtreeviz facebook_business

In [7]:
%load_ext autoreload
%autoreload 2

In [111]:
pd.read_csv('outs/ma.csv').sort_values('disthash').cost_per_completion

0      45.454545
1     142.857143
2      76.923077
3      58.823529
4     100.000000
         ...    
75     66.666667
76     37.037037
77     66.666667
78    125.000000
79     47.619048
Name: cost_per_completion, Length: 80, dtype: float64

In [ ]:
pd.read_csv('outs/dist_info.csv').sort_values('clusterid')

In [2]:
from environs import Env
env = Env()
env.read_env('.env-upswell-vlab', override=True)

In [ ]:
from adopt.malaria import get_conf, get_df, lookalike, get_ad_token
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState

# cnf = get_conf(env)
# token = get_ad_token(cnf["survey_user"], cnf["chatbase"])

# state = CampaignState(env, token)
# m = Marketing(env, state)

# TODO: make new strata format conf parse
# TODO: make get_df work with new strata format
# df = get_df(cnf)

In [6]:
cnf = { 'chatbase':  {
        "db": "chatroach",
        "user": "chatreader",
        "host": "localhost",
        "port": "5432",
        "password": None,
    }}

In [18]:
import pandas as pd
import geopandas as gpd

def get_dist_info(path):
    geod = gpd.read_file(path)
    district_info = geod.groupby('disthash').apply(lambda df: df.iloc[0][['disthash', 'tot_p']]).reset_index(drop=True)
    return district_info

def load_cities(path):
    cities = pd.read_csv(path)
    cities = cities[cities.rad >= 1.0]
    return cities

dist_info = get_dist_info('outs/cluster.shp')
cities = load_cities('output/cities-22.csv')
cities = cities.merge(dist_info, how='left', on='disthash')

districts = cities.groupby('distname') \
                  .head(1) \
                  .reset_index(drop=True) \
                  .drop(columns=['rad', 'lng', 'lat', 'distcode', 'id'])

In [19]:
from adopt.responses import get_forms
from datetime import datetime

survey_user = '10383123-9fb1-429b-8312-447c1b54b17a'
# shortcodes = [ 'follow1eng', 'follow1hin']
shortcodes = [ 'baselineeng', 'baselinehin', 'baselineodi']

timestamp = datetime(2020, 9, 1)

forms = list(get_forms(survey_user, shortcodes, timestamp, cnf['chatbase']))

In [20]:
questions =[('malaria_now', '58583e72-7036-4c74-9ebd-179598cba59f'),
             ('net_lastnight', '1f812d37-a7a5-400d-8a31-145fbb49f577'),]

In [21]:
questions = [
    ('dist_medical', 'd7573919-8a7e-457f-9a1d-1f8c389127a7'),
    ('education', 'e40fa1c6-13a1-4a02-91cd-0eaade11864d'),
    ('owns_net', 'bd4802c6-7bdb-40f0-aac1-18cc6df7da6e'),
    ('malaria_incidence', 'f6e69027-97cc-494e-8d52-318b75047e23'),
    ('malaria_now', 'dad23031-8468-4900-89cc-d01841d8b660'),
    ('net_lastnight', 'b5b1ff58-c8df-4890-9b1c-0cd40ce6edc0'),
    ('home', '4d0ae478-3893-4b46-ab39-d6848c69245d'), 
    ('occupation', '4fc929c7-132d-49b1-a164-515e5cc9064f')]

In [22]:
from adopt.responses import get_response_df
from datetime import datetime, timezone
from adopt.clustering import only_target_users
import pandas as pd
from adopt.clustering import add_res_cols
from adopt.forms import response_translator
from adopt.malaria import shape_df
import re


def filter_time(df, min_date, max_date):
    min_date = datetime(*min_date, tzinfo=timezone.utc)
    max_date = datetime(*max_date, tzinfo=timezone.utc)

    users = df[(df.timestamp > min_date) & (df.timestamp < max_date)].userid.unique()
    return df[df.userid.isin(users)]

def ref_translation(eng_form, other_form):
    vals = [(a['ref'], b['ref']) for a, b 
            in zip(eng_form['fields'], other_form['fields'])]
    lookup = dict(vals)
    rev_lookup = {v:k for k,v in lookup.items()}
    return lookup, rev_lookup


def get_filtered_responses(survey_user, eng_form, other_form, shortcodes, questions, db_cnf):
    ref_lookup, rev_ref_lookup = ref_translation(eng_form, other_form)
    refs = [ref_lookup[r] for _, r in questions]
    responses = get_response_df(survey_user, shortcodes, refs, db_cnf)

    _, last_ref = questions[-1]
    
    answered = responses[responses.question_ref == last_ref].userid.unique()    

    return responses[responses.userid.isin(answered)].reset_index(drop=True)

def malaria_prob(groupby, key='malaria'):
    d = groupby[key].value_counts(normalize=True)
    d.name = 'count'
    d = d.reset_index(level=-1)
    d = d[d[key] == True].drop(columns=key)
    return d


def stats(df):
    targets = df.kutcha.sum()
    tot = df.shape[0]
    non_target = tot - targets
    return pd.Series([targets/tot, targets, non_target, tot], index=['perc', 'target', 'non_target', 'tot'])


def col_translators(a, b, questions):
    lookup, _ = ref_translation(a, b)

    refs = [ref for _, ref in questions]
    ts = { qt['ref']: response_translator(q, qt)
           for qt, q in zip(a['fields'], b['fields'])
           if qt['ref'] in refs}

    return [(n, lookup[ref], ts[ref]) for n, ref in questions]


In [23]:
# filtered_res = get_filtered_responses(survey_user, forms[0], forms[1], ['follow1hin'], questions, cnf['chatbase'])
filtered_res = get_filtered_responses(survey_user, forms[0], forms[1], ['baselinehin', 'baselinehinexc'], questions, cnf['chatbase'])

col_names = col_translators(forms[0], forms[1], questions)

In [24]:

def shape_df(df):
    df = df \
        .groupby(['surveyid', 'shortcode']) \
        .apply(lambda df: df.pivot('userid', 'question_ref', 'response')) \
        .reset_index()

    df['timestamp'] = df['md:startTime'].map(lambda x: datetime.fromtimestamp(x/1000,
                                                                              tz=timezone.utc))

    # Clean anyone who answered multiple surveys in this group of shortcodes
    # in theory should only be testers.
    # Keep the lastest survey they took.
    return df \
        .sort_values('timestamp') \
        .drop_duplicates(['userid'], keep='last')

In [25]:
r = shape_df(filtered_res)

In [ ]:
r[r.timestamp <= datetime(2021, 4, 30, tzinfo=timezone.utc)].userid.unique().shape

In [ ]:
r.surveyid.unique()

In [ ]:
rr = add_res_cols(col_names, r) \
    .reset_index(drop=True) \
    .dropna()

In [99]:
rr.userid.unique().shape

(23084,)

In [27]:
dd = rr.merge(districts, left_on='md:clusterid', right_on='disthash')
qrefs = [c for c in dd.columns if len(c) > 35]
dd = dd.drop(columns=qrefs)

dd['under_net'] = dd.net_lastnight == 'Yes'
dd['malaria'] = dd.malaria_incidence == 'Yes'
dd['malaria_now'] = dd.malaria_now == 'Yes'
dd['kutcha'] = dd.home == 'Kutcha (made of mud, tin, straw)'
dd['pucca'] = dd.home == 'Pucca (have cement/brick wall and floor'
dd['university'] = dd.education == 'University degree or higher'
dd['unemployed'] = dd.occupation == 'Unemployed'
dd['disthash'] = dd['md:clusterid']

In [31]:
dd.userid.unique().shape

(22978,)

In [19]:
perc = dd.groupby('disthash').apply(stats).reset_index()

In [76]:
# treatment_assignment = pd.read_csv("outs/ma-with-treatment.csv")[['disthash', 'treatment']]
# dd.merge(treatment_assignment).to_csv('outs/individual-with-treatment.csv', index=False)

In [53]:
filter_time(dd, (2020,8,24), (2020,8,25)).shape

(160, 19)

In [54]:
filter_time(dd, (2020,8,25), (2020,8,26)).shape

(204, 19)

In [55]:
filter_time(dd, (2020,8,26), (2020,8,27)).shape

(57, 19)

In [56]:
filter_time(dd, (2020,8,27), (2020,8,28)).shape

(223, 19)

In [57]:
filter_time(dd, (2020,8,28), (2020,8,29)).shape

(1072, 19)

In [58]:
filter_time(dd, (2020,8,29), (2020,8,30)).shape

(383, 19)

In [59]:
filter_time(dd, (2020,8,30), (2020,9,1)).shape

(212, 19)

In [20]:
from adopt.clustering import get_budget_lookup
from adopt.malaria import window, days_left, get_df, get_cluster_from_adset
from adopt.facebook.state import CampaignState, BudgetWindow

# w = BudgetWindow(datetime(2020,8,10), datetime.now())

w = window(96)
state = CampaignState(env, w)

spend = {get_cluster_from_adset(n): i
         for n, i in state.spend.items()}

In [21]:
budget_lookup, prices = get_budget_lookup(df,
                                          stratum,
                                          float('inf'),
                                          10000,
                                          84,
                                          6,
                                          w,
                                          spend, 
                                          return_price=True)

In [22]:
bb = pd.Series(budget_lookup) / 100

In [ ]:
with pd.option_context('display.max_rows', None):
    display(bb.sort_values()[:80])

In [23]:
bb.sort_values()[:80].sum() / 70 * 12

1460.1925714285717

In [24]:
def add_budget(r, lookup, key='budget'):
    r[key] = lookup.get(r['disthash'], None)
    return r

perc = perc.apply(lambda r: add_budget(r, budget_lookup), 1)
perc = perc.apply(lambda r: add_budget(r, prices, 'price'), 1)

In [25]:
mn_lookup = dd.groupby('disthash').apply(lambda df: pd.Series({k: df[k].mean() for k in ['malaria_now', 'malaria']})).reset_index()
perc = perc.merge(mn_lookup)

In [26]:
excluded = cities[cities.creative_group == 'exclusions'].disthash.unique()
perc['exclusion'] = perc.disthash.isin(excluded)

In [27]:
audienced = cities[cities.include_audience == True].disthash.unique()
perc['audienced'] = perc.disthash.isin(audienced)

In [28]:
perc = perc.merge(dd.groupby('disthash').under_net.mean().reset_index(), how='left')

In [29]:
saturated = (only_target_users(df, surveys, 'target_questions')
             .groupby('md:clusterid')
             .userid.count() >= 250) \
             .reset_index() \
             .pipe(lambda df: df[df.userid == True])['md:clusterid']

In [30]:
# cities.loc[cities.disthash == 'c12daf4e', 'creative_group'] = 'exclusions'

In [31]:
final_clusters = pd.read_csv('outs/final-clusters.csv').disthash

In [32]:
perc['final'] = perc.disthash.isin(final_clusters)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.expand_frame_repr', False):
    display(perc.sort_values('tot'))

In [63]:
treatment_assignment = pd.read_csv("outs/ma-with-treatment.csv")[['disthash', 'treatment']]
# dd.merge(treatment_assignment).to_csv('outs/individual-with-treatment.csv', index=False)

In [70]:
perc = dd.groupby('disthash').apply(lambda df: pd.Series({'tot': df.shape[0]})).reset_index()
treatment_assignment = pd.read_csv("outs/ma-with-treatment.csv")[['disthash', 'treatment']]
perc = perc.merge(treatment_assignment)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.expand_frame_repr', False):
    display(perc.sort_values('tot'))

In [28]:
def matching_stats(df):
    vals = {
        'kutchas': df.kutcha.mean(),
        'puccas': df.pucca.mean(),
        'university': df.university.mean(),
        'unemployed': df.unemployed.mean(),
        'malaria': df.malaria.mean(),
        'malaria_now': df.malaria_now.mean(),
        'under_net': df.under_net.mean(),
        'population': df.tot_p.max()
    }

    return pd.Series(vals)


def get_cluster_from_adset(adset_name: str) -> str:
    pat = r"(?<=vlab-)\w+"
    matches = re.search(pat, adset_name)
    if not matches:
        raise Exception(f"Cannot extract cluster id from adset: {adset_name}")

    return matches[0]

def prep_facebook_data(path, dd, final_clusters):
    capture_count = dd[(dd.timestamp > datetime(2020,7,23,tzinfo=timezone.utc)) & 
                       (dd.timestamp < datetime(2020,7,24,tzinfo=timezone.utc))] \
        .groupby('disthash') \
        .userid.count() \
        .reset_index() \
        .rename(columns={'userid': 'completed_survey'})

    df = pd.read_csv(path)
    df['disthash'] = df['Ad Set Name'].map(get_cluster_from_adset)
    df = df.merge(capture_count, how='left')
    df = df[df.disthash.isin(final_clusters)].reset_index(drop=True)

    spend = 'Amount Spent (INR)'
    df['cost_per_completion'] = df.apply(lambda r: r[spend] / r.completed_survey, 1)
    df = df.rename(columns = {'CTR (All)': 'CTR', 
                              'Cost per Results': 'cost_per_message', 
                              'CPM (Cost per 1,000 Impressions) (INR)': 'CPM'})

    return df[['disthash', 'cost_per_completion', 'cost_per_message', 'CTR', 'CPM']]

In [29]:
dd = dd.rename(columns={'tot_p_x': 'tot_p'})
final_clusters = pd.read_csv('outs/final-clusters.csv').disthash

fb = prep_facebook_data('outs/fb-export-vlab-1.csv', dd, final_clusters)

ma = dd.groupby('disthash').apply(matching_stats).reset_index()
ma = ma[ma.disthash.isin(final_clusters)]
ma.loc[ma.population.isna(), 'population'] = ma.population.quantile(.2) # random data filling :/

ma = ma.merge(fb)

In [30]:
ma.sort_values('disthash').cost_per_completion

0      45.454545
1     142.857143
2      76.923077
3      55.555556
4      90.909091
         ...    
75     66.666667
76     37.037037
77     66.666667
78    125.000000
79     45.454545
Name: cost_per_completion, Length: 80, dtype: float64

In [ ]:
perc['saturated'] = perc.disthash.isin(saturated)

perc_stats = perc[['disthash', 'tot', 'audienced', 'exclusion', 'price', 'saturated']].rename(columns={'tot': 'current_total'})

treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
perc = perc.merge(treatment_assignment, how='left')

ma.merge(treatment_assignment).merge(perc_stats).to_csv('outs/ma-3.csv', index=False)

In [ ]:
# perc.groupby('treatment').under_net.mean()
# perc['nn'] = perc.apply(lambda r: max(0, (250 - r.tot)) * r.under_net, 1)
# perc.groupby('treatment').nn.sum()
# dd.merge(treatment_assignment).groupby('treatment').under_net.sum()

In [ ]:
# saturated = perc[perc.non_target > 175].disthash
# audienced = perc[(perc.perc < .3) & ((perc.disthash.isin(saturated) | (perc.price < 30000)))].disthash
# cities['creative_group'] = 'hindi'
# cities.loc[cities.disthash.isin(saturated), 'creative_group'] = 'exclusions'
# cities['include_audience'] = False
# cities.loc[cities.disthash.isin(audienced), 'include_audience'] = True

In [ ]:
tot = 196952 + 17682 + 40222 + 89997 + 113061 + 50145 + 141910 + 4255509 + 46000 

In [ ]:
tot / 70